# Quick test of Ollama and the hopper H200
- [LangChain Ollama Docs](https://python.langchain.com/docs/guides/development/local_llms/#quickstart)
- [Multimodal Ollama Cookbook](https://docs.llamaindex.ai/en/stable/examples/multi_modal/ollama_cookbook/)
- [Multi-Agent DSPy Programs: Bootstrapping & Aggregating Multiple ReAct Agents](https://github.com/stanfordnlp/dspy/blob/main/examples/agents/multi_agent_llama3.ipynb)
- [Don’t RAG on Knowledge Graphs(Or Do) Benchmarking: Finally Building a Knowledge Graph – Part Two](https://www.dmlbl.com/technical_blog/knowledge-graph-rag-benchmark-2/)
- [Rustworkx for visualization](https://www.rustworkx.org/index.html)
- [Paco's ER Blog Post](https://neo4j.com/developer-blog/entity-resolved-knowledge-graphs/)

## Currentlly running LLMs on the Hopper H200

| NAME                          |     ID         |  SIZE   |     MODIFIED        |
|-------------------------------|----------------|---------|---------------------|
| all-minilm:latest             |  1b226e2802db  |  45 MB  |  About a minute ago |
| command-r-plus:latest         |  c9c6cc6d20c7  |  59 GB  |  3 minutes ago      |
| dbrx:latest                   |  36800d8d3a28  |  74 GB  |  19 minutes ago     |
| dolphin-llama3:latest         |  613f068e29f8  |  4.7 GB | About a minute ago  |
| dolphin-mixtral:latest        |  cfada4ba31c7  |  26 GB  | 2 minutes ago       |
| gemma:latest                  |  a72c7f4d0a15  |  5.0 GB | 31 seconds ago      |
| llama3:70b-instruct           |  bcfb190ca3a7  |  39 GB  | 2 days ago          |
| llama3:latest                 |  71a106a91016  |  4.7 GB | 2 days ago          |
| llava:latest                  |  8dd30f6b0cb1  |  4.7 GB | 10 seconds ago      |
| mixtral:8x22b                 |  bf88270436ed  |  79 GB  | 23 minutes ago      |
| mxbai-embed-large:latest      |  468836162de7  |  669 MB | About a minute ago  |
| nomic-embed-text:latest       |  0a109f422b47  |  274 MB |  57 seconds ago     |
| snowflake-arctic-embed:latest |  21ab8b9b0545  |  669 MB | About a minute ago  |
| starcoder2:latest             |  f67ae0f64584  |  1.7 GB | About a minute ago  |
| wizardlm2:latest              |  c9b1aff820f2  |  4.1 GB | 3 minutes ago       |

## SSH Config
Place in `~/.ssh/config`:
```
Host hopper
  Hostname gh200-001.crc.nd.edu
  User <your user name>
  LocalForward 8888 127.0.0.1:8888
  LocalForward 11434 127.0.0.1:11434
```

In [1]:
import ollama
response = ollama.chat(model='phi3:instruct', messages=[
  {
    'role': 'user',
    'content': 'Why do ducks have webbed feet?',
  },
])
print(response['message']['content'])

 Ducks have webbed feet primarily as an adaptation for efficient swimming. The webbing between their toes increases the surface area of their feet, making them excellent paddles in water. This feature helps ducks propel themselves with greater ease and speed while swimming. Additionally, this specialized structure aids in steering and maneuverability when navigating through various aquatic environments. Beyond swimming efficiency, webbed feet can also help ducks to walk on slippery surfaces like ice more effectively.

Here are some other benefits of the webbed feet in ducks:

1. Diving capabilities: Webbed feet assist ducks in diving deeper and staying underwater for longer periods, which is crucial for feeding and avoiding predators.
2. Ground stability: Although their primary function isn't ground walking, the webbed structure helps with balance and provides a certain level of traction on soft or uneven terrain when ducks need to walk briefly out of the water.
3. Foraging efficiency:

In [3]:
# Multimodal model on Hopper: llava-phi3, llava:34b, moondream, llava-llama3
from langchain_community.llms import Ollama
mmllm = Ollama(base_url="http://localhost:11434", model="llava-phi3")
#mmllm = ollama.chat(model="llava-phi3")

from PIL import Image
import base64
from io import BytesIO

def convert_to_base64(pil_image: Image):
    buffered = BytesIO()
    pil_image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def load_image(image_path: str):
    pil_image = Image.open(image_path)
    image_b64 = convert_to_base64(pil_image)
    print("Loaded image successfully!")
    return image_b64

In [6]:
image_b64 = load_image("./contextual_test.png")
resp = mmllm.invoke("What's in the image?", images=[image_b64])
print(resp)

Loaded image successfully!
The image captures a moment between two men standing side by side, each wearing a black t-shirt. The man on the left is smiling and has his arm casually draped around the shoulder of the other man on the right. 

Both shirts are identical in color but bear different phrases written in white text. On the shirt of the man to the left, the phrase "I didn't invent the internet" is printed. The man on the right, with his arm around his companion, has a shirt that says "I did not invent the Web". 

The background behind them is a gradient of green and blue colors, providing a contrasting backdrop to their black t-shirts. The phrase on each shirt is clearly visible, indicating a humorous acknowledgment of these two iconic technological advancements often attributed to Tim Berners-Lee.


Using the Ollama Chat Library directly:


In [4]:
import ollama
with open('./contextual_test.png', 'rb') as file:
  response = ollama.chat(
    model='llava-phi3',
    messages=[
      {
        'role': 'user',
        'content': 'What is strange about this image?',
        'images': [file.read()],
      },
    ],
  )
print(response['message']['content'])

The strangeness of the image lies in the fact that both men are wearing t-shirts with phrases written on them, but the content of these phrases is humorous or satirical and does not seem to be related to their physical presence. The man on the left has a t-shirt that says "I didn't invent the internet", while the man on the right has one that says "I didn't invent the web". These statements are factually incorrect, as they imply ownership or invention of technology platforms that were actually developed by others. This unusual choice of phrasing adds a layer of humor and irony to the image.
